In [1]:
import cv2
from cvzone.PoseModule import PoseDetector
import math
import numpy as np
import time
import pickle
import mediapipe as mp
# Drawing helpers
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose
import pandas as pd

import os, sys
sys.path.append(os.path.abspath(".."))
from utils.common import load_model, save_model
import warnings
warnings.filterwarnings('ignore')

In [2]:
def rescale_frame(frame, percent=50):
    '''
    Rescale a frame to a certain percentage compare to its original frame
    '''
    width = int(frame.shape[1] * percent/ 100)
    height = int(frame.shape[0] * percent/ 100)
    dim = (width, height)
    return cv2.resize(frame, dim, interpolation =cv2.INTER_AREA)

In [3]:
IMPORTANT_LMS = [
    "NOSE",
    "LEFT_SHOULDER",
    "RIGHT_SHOULDER",
    "LEFT_HIP",
    "RIGHT_HIP",
    "LEFT_KNEE",
    "RIGHT_KNEE",
    "LEFT_ANKLE",
    "RIGHT_ANKLE"
]

# Tạo các cột cho dữ liệu đầu vào
HEADERS = ["label"]
for landmark in IMPORTANT_LMS:
    for dim in ['x', 'y', 'z']:
        HEADERS.append(f"{landmark.lower()}_{dim}")
        
# IMPORTANT_LMS = [element.lower() for element in IMPORTANT_LMS]

def extract_amd_recalculate_landmarks(pose_landmarks):
    """
    Tịnh tiến thân người vào giữa bức hình, đồng thời dời lại trục toạ độ
    """
    hip_center_x = float((pose_landmarks[23].x + pose_landmarks[24].x) / 2)
    hip_center_y = float((pose_landmarks[23].y + pose_landmarks[24].y) / 2)

    new_center = (0.5, 0.5)
    delta_x = new_center[0] - hip_center_x
    delta_y = new_center[1] - hip_center_y

    data = []
    for landmark in IMPORTANT_LMS:
        # Lấy ra id của key point trên cơ thể người
        key_point_id = mp_pose.PoseLandmark[landmark].value

        key_point = pose_landmarks[key_point_id]
        key_point.x += delta_x - 0.5
        key_point.y += delta_y -0.5
        data.append([key_point.x, key_point.y, key_point.z])

    return np.array(data).flatten().tolist()

In [4]:
import os
import soundfile as sf

# Khởi tạo dictionary để lưu các đối tượng audio
error_types_audio = {}

# Thư mục chứa các file âm thanh
folder_path = "audios_side"
current_path = os.getcwd()

# Duyệt qua các file trong thư mục
for filename in os.listdir(folder_path):
    file_path = os.path.join(current_path, folder_path, filename)
    if os.path.isfile(file_path):
        data, samplerate = sf.read(file_path)
        filename = filename.replace(".mp3", "")
        error_types_audio[filename] = (data, samplerate)

# In dictionary sau khi lưu
print(error_types_audio)


{'bend_forward': (array([-1.10624029e-07, -5.38439917e-06,  9.93211415e-06, ...,
       -3.80218917e-05, -6.26517358e-05,  1.10427907e-04]), 16000), 'bend_forward_deep_squat': (array([-6.28902569e-07,  6.16414763e-06,  1.20748527e-05, ...,
       -3.55165503e-05, -7.22415280e-06,  2.46893651e-05]), 16000), 'bend_forward_knees_straight': (array([ 8.11194768e-05,  6.71087619e-05,  4.36834671e-05, ...,
        6.21243544e-06, -1.93064734e-05, -3.67521388e-05]), 16000), 'bend_forward_lift_hips': (array([ 2.71227946e-05, -1.44367477e-05, -6.49801950e-06, ...,
        1.02917884e-05,  1.15255061e-05, -5.73748694e-05]), 16000), 'deep_squat': (array([ 1.83161956e-06,  1.75427090e-07,  1.26069672e-05, ...,
        3.36799440e-06, -1.50981277e-05, -4.83399981e-06]), 16000), 'deep_squat_knees_straight': (array([ 4.18786931e-06, -5.67916277e-06, -3.47432388e-06, ...,
       -1.16236060e-05,  5.60990566e-06,  1.26066938e-04]), 16000), 'deep_squat_lift_hips': (array([ 8.19807883e-06, -7.24470647e-06

In [5]:
is_playing = False

In [6]:
import threading
import sounddevice as sd

def play_audio(data, samplerate):
    global is_playing
    is_playing = True
    sd.play(data, samplerate)
    sd.wait()
    is_playing = False

# Hàm bắt đầu một luồng để phát âm thanh
def start_audio_thread(data, samplerate):
    threading.Thread(target=play_audio, args=(data, samplerate,), daemon=True).start()

In [7]:
def get_class(encode_label: float):
    return {
        0: "D",
        1: "M",
        2: "U",
    }.get(encode_label, "Unknown")

In [8]:
from utils.common import load_model

model = load_model("./best_models/SVC_model_side.pkl")
input_scaler = load_model("./best_models/input_scaler.pkl")

## Ver original

In [9]:
video_path = 'squat.mp4'
cap = cv2.VideoCapture(video_path)
detector = PoseDetector(detectionCon=0.7, trackCon=0.7)

# Creating Angle finder class
class angleFinder:
    def __init__(self, lmlist, p1, p2, p3, p4, p5, p6, p7, p8, drawPoints):
        self.lmlist = lmlist
        self.p1 = p1
        self.p2 = p2
        self.p3 = p3
        self.p4 = p4
        self.p5 = p5
        self.p6 = p6
        self.p7 = p7
        self.p8 = p8
        self.drawPoints = drawPoints

    # finding angles
    def angle(self):
        if len(self.lmlist) != 0:
            hipLeft = self.lmlist[self.p1][:2]
            kneeLeft = self.lmlist[self.p2][:2]
            ankleLeft = self.lmlist[self.p3][:2]
            hipRight = self.lmlist[self.p4][:2]
            kneeRight = self.lmlist[self.p5][:2]
            ankleRight = self.lmlist[self.p6][:2]
            leftShoulder = self.lmlist[self.p7][:2]
            rightShoulder = self.lmlist[self.p8][:2]

            if len(hipLeft) >= 2 and len(kneeLeft) >= 2 and len(ankleLeft) >= 2 and len(hipRight) >= 2 and len(kneeRight) >= 2 and len(
                    ankleRight) >= 2 and len(leftShoulder) >= 2 and len(rightShoulder) >= 2:
                x1, y1 = hipLeft[:2]
                x2, y2 = kneeLeft[:2]
                x3, y3 = ankleLeft[:2]
                x4, y4 = hipRight[:2]
                x5, y5 = kneeRight[:2]
                x6, y6 = ankleRight[:2]
                x7, y7 = leftShoulder[:2]
                x8, y8 = rightShoulder[:2]

                vertical_line_angle = 90  # Góc của đường thẳng đứng so với trục x

                # calculating angle for left and right hands
                leftHandAngle = math.degrees(math.atan2(y1 - y2, x1 - x2) - math.atan2(y1 - y2, 0))
                rightHandAngle = math.degrees(math.atan2(y4 - y5, x4 - x5) - math.atan2(y4 - y5, 0))

                leftBackAngle = math.degrees(math.atan2(y1 - y2, 0) - math.atan2(y7 - y1, x7 - x1))
                rightBackAngle = math.degrees(math.atan2(y4 - y5, 0) - math.atan2(y8 - y4, x8 - x4))

                p2_p3_angle = math.degrees(math.atan2(y3 - y2, x3 - x2))
                p5_p6_angle = math.degrees(math.atan2(y6 - y5, x6 - x5))
                # Tính góc giữa đường thẳng đứng và đường thẳng nối giữa đầu gối và mắt cá chân
                leftkneeAngleLineAngle = abs(vertical_line_angle - p2_p3_angle)
                rightkneeAngleLineAngle = abs(vertical_line_angle - p5_p6_angle)

                # print (f"leftHandAngle: {leftHandAngle}")
                # print (f"rightHandAngle: {rightHandAngle}")
                # print (f"leftBackAngle: {leftBackAngle}")
                # print (f"rightBackAngle: {rightBackAngle}")
                # print (f"leftkneeAngleLineAngle: {leftkneeAngleLineAngle}")
                # print (f"rightkneeAngleLineAngle: {rightkneeAngleLineAngle}")
                

                leftHandAngle = int(np.interp(leftHandAngle, [0, 95], [0, 100])) # Ánh xạ sang từ 0-95 về 0-100 vì góc càng nhỏ thì càng tiến về trạng thái s3
                rightHandAngle = int(np.interp(rightHandAngle, [0, 95], [0, 100])) # Ánh xạ sang từ 0-95 về 0-100 vì góc càng nhỏ thì càng tiến về trạng thái s3
                leftBackAngle = int(np.interp(leftBackAngle, [0, 95], [0, 100])) # Ánh xạ sang từ 0-95 về 0-100 vì góc càng nhỏ thì càng tiến về trạng thái s3
                rightBackAngle = int(np.interp(rightBackAngle, [0, 95], [0, 100])) # Ánh xạ sang từ 0-95 về 0-100 vì góc càng nhỏ thì càng tiến về trạng thái s3

                # drawing circles and lines on selected points
                if self.drawPoints:
                    cv2.circle(img, (x1, y1), 2, (0, 255, 0), 6)          
                    cv2.circle(img, (x2, y2), 2, (0, 255, 0), 6)           
                    cv2.circle(img, (x3, y3), 2, (0, 255, 0), 6)           
                    cv2.circle(img, (x4, y4), 2, (0, 255, 0), 6)           
                    cv2.circle(img, (x5, y5), 2, (0, 255, 0), 6)           
                    cv2.circle(img, (x6, y6), 2, (0, 255, 0), 6)            
                    cv2.circle(img, (x7, y7), 2, (0, 255, 0), 6)           
                    cv2.circle(img, (x8, y8), 2, (0, 255, 0), 6)

                    cv2.line(img, (x1, y1), (x2, y2), (0, 0, 255), 2)
                    cv2.line(img, (x2, y2), (x3, y3), (0, 0, 255), 2)
                    cv2.line(img, (x4, y4), (x5, y5), (0, 0, 255), 2)
                    cv2.line(img, (x5, y5), (x6, y6), (0, 0, 255), 2)
                    cv2.line(img, (x1, y1), (x4, y4), (0, 0, 255), 2)
                    cv2.line(img, (x1, y1), (x7, y7), (0, 0, 255), 2)
                    cv2.line(img, (x4, y4), (x8, y8), (0, 0, 255), 2)

                return [leftHandAngle, rightHandAngle, leftBackAngle, rightBackAngle, leftkneeAngleLineAngle, rightkneeAngleLineAngle]

# defining some variables
counter = 0
direction = 0
error1 = False
error2 = False
error3 = False
error4 = False
error5 = False

error1_start_time = None
error2_start_time = None
error3_start_time = None
error4_start_time = None
error5_start_time = None

with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while True:
        ret, img = cap.read()
        # img = cv2.resize(img, (640, 480))
        img = rescale_frame(img, percent=20)

        if not ret:
            print("Ignoring empty camera frame.")
            break
                
        # resize frame để tăng tốc độ xử lý
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        results = pose.process(img)

        if not results.pose_landmarks:
            print("No human found")
            continue

        # Cần khôi phục lại màu gốc của ảnh
        img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)

        key_points = extract_amd_recalculate_landmarks(results.pose_landmarks.landmark)
        X = pd.DataFrame([key_points], columns=HEADERS[1:])
        X = input_scaler.transform(X)

        predicted_class = model.predict(X)[0]
        predicted_class = get_class(model.predict(X)[0])

        # putting predicted class on the screen
        cv2.putText(img, f"State: {predicted_class}", (10, 420), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)


        detector.findPose(img, draw=0)
        lmList, bboxInfo = detector.findPosition(img, bboxWithHands=0, draw=False)
        
        angle1 = angleFinder(lmList, 23, 25, 27, 24, 26, 28, 11, 12, drawPoints=True)
        hands = angle1.angle()
        left, right, leftBackAngle, rightBackAngle, leftkneeAngleLineAngle, rightkneeAngleLineAngle = hands[0:]

        error = []

        # Set lại giá trị ban đầu
        error1 = False
        error2 = False
        error3 = False
        error4 = False

        # Counting number of squat
        if left > 75 and right > 75:
            if direction == 0:
                counter += 0.5
                direction = 1
        if left <= 70 and right <= 70:
            if direction == 1:
                counter += 0.5
                direction = 0

        current_time = time.time()

        if leftBackAngle >= 45 and rightBackAngle >= 45:
            error1 = True
            error.append("bend_forward")
            if error1_start_time is None:
                error1_start_time = current_time
        elif error1_start_time is not None:
            error1_start_time = None

        if left >= 95 and right >= 95:
            error4 = True
            error.append("deep_squat")
            if error4_start_time is None:
                error4_start_time = current_time
        elif error4_start_time is not None:
            error4_start_time = None

        if leftkneeAngleLineAngle >= 30 and rightkneeAngleLineAngle >= 30:
            error3 = True
            error.append("knees_straight")
            if error3_start_time is None:
                error3_start_time = current_time
        elif error3_start_time is not None:
            error3_start_time = None

        if leftBackAngle >= 30 and rightBackAngle >= 30 and left <= 80 and right <= 80:
            error2 = True
            error.append("lift_hips")
            if error2_start_time is None:
                error2_start_time = current_time
        elif error2_start_time is not None:
            error2_start_time = None

        error = ", ".join(error)

        # putting scores on the screen
        cv2.rectangle(img, (0, 0), (120, 120), (255, 0, 0), -1)
        cv2.putText(img, str(int(counter)), (1, 70), cv2.FONT_HERSHEY_SCRIPT_SIMPLEX, 1.6, (0, 0, 255), 6)

        # Draw errors
        if error1 and error1_start_time and (current_time - error1_start_time) < 1000:
            cv2.rectangle(img, (380, 60), (630, 100), (64, 64, 204), -1)
            cv2.putText(img, "Bend Forward", (390, 80), cv2.FONT_HERSHEY_TRIPLEX , 0.7, (255, 255, 230), 3)
        if error2 and error2_start_time and (current_time - error2_start_time) < 1000:
            cv2.rectangle(img, (380, 110), (630, 150), (64, 64, 204), -1)
            cv2.putText(img, "Lower one's hips", (390, 130), cv2.FONT_HERSHEY_TRIPLEX, 0.7, (255, 255, 230), 3)
        if error3 and error3_start_time and (current_time - error3_start_time) < 1000:
            cv2.rectangle(img, (380, 160), (630, 200), (64, 64, 204), -1)
            cv2.putText(img, "Knee falling over toes", (390, 180), cv2.FONT_HERSHEY_TRIPLEX, 0.6, (255, 255, 230), 3)
        if error4 and error4_start_time and (current_time - error4_start_time) < 1000:
            cv2.rectangle(img, (380, 210), (630, 250), (204, 122, 0), -1)
            cv2.putText(img, "Deep squats", (390, 230), cv2.FONT_HERSHEY_TRIPLEX, 0.7, (255, 255, 230), 3)

        parts = error.split(",")[:2]
        # Nối các phần lại với nhau
        error = ", ".join(parts)
        error_type = error.replace(",  ", "_").replace(" ", "_").lower()
        print("Error type: " + error_type)
        if error_type in error_types_audio and not is_playing:
            data, samplerate = error_types_audio[error_type]
            start_audio_thread(data, samplerate)

        # Converting values for rectangles
        leftval = np.interp(left, [0, 100], [400, 200])
        rightval = np.interp(right, [0, 100], [400, 200])

        # For color changing
        value_left = np.interp(left, [0, 100], [0, 100])
        value_right = np.interp(right, [0, 100], [0, 100])

        # Drawing right rectangle and putting text
        cv2.putText(img, 'R', (24, 195), cv2.FONT_HERSHEY_DUPLEX, 1, (255, 0, 0), 5)
        cv2.rectangle(img, (8, 200), (50, 400), (0, 255, 0), 5)
        cv2.rectangle(img, (8, int(rightval)), (50, 400), (255, 0, 0), -1)

        # Drawing right rectangle and putting text
        cv2.putText(img, 'L', (604, 195), cv2.FONT_HERSHEY_DUPLEX, 1, (255, 0, 0), 5)
        cv2.rectangle(img, (582, 200), (632, 400), (0, 255, 0), 5)
        cv2.rectangle(img, (582, int(leftval)), (632, 400), (255, 0, 0), -1)

        # Tô màu đỏ khi góc đạt đến trạng thái s3
        if value_left > 75:
            cv2.rectangle(img, (582, int(leftval)), (632, 400), (0, 0, 255), -1)

        if value_right > 75:
            cv2.rectangle(img, (8, int(rightval)), (50, 400), (0, 0, 255), -1)

        cv2.imshow("Image", img)

        # Nhấn q để thoát
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

Error type: knees_straight
Error type: knees_straight
Error type: knees_straight
Error type: 
Error type: 
Error type: 
Error type: 
Error type: 
Error type: 
Error type: 
Error type: 
Error type: 
Error type: 
Error type: 
Error type: 
Error type: 
Error type: 
Error type: 
Error type: 
Error type: 
Error type: 
Error type: 
Error type: 
Error type: 
Error type: 
Error type: 
Error type: 
Error type: 
Error type: 
Error type: 
Error type: 
Error type: 
Error type: 
Error type: 
Error type: 
Error type: 
Error type: 
Error type: 
Error type: 
Error type: 
Error type: 
Error type: 
Error type: 
Error type: 
Error type: 
Error type: 
Error type: 
Error type: 
Error type: 
Error type: 
Error type: 
Error type: 
Error type: 
Error type: 
Error type: 
Error type: 
Error type: 
Error type: 
Error type: 
Error type: 
Error type: 
Error type: 
Error type: 
Error type: 
Error type: 
Error type: 
Error type: knees_straight
Error type: knees_straight
Error type: knees_straight
Error type: knees_s